In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import randint, norm
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.svm import SVR

In [2]:
spotify = pd.read_csv('./assets/spotify_songs.csv')

spotify = spotify.drop(columns=["track_id", "track_name", "track_artist", "track_album_id", "track_album_name", "playlist_name", "playlist_id"])

spotify = pd.get_dummies(data=spotify, columns=["playlist_genre", "playlist_subgenre"])

spotify["track_album_release_date"] = spotify["track_album_release_date"].str.split("-").str[0]

spotify["track_album_release_date"] = spotify["track_album_release_date"].astype(float)

spotify.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32821 entries, 0 to 32820
Data columns (total 44 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   track_popularity                             32821 non-null  int64  
 1   track_album_release_date                     32821 non-null  float64
 2   danceability                                 32821 non-null  float64
 3   energy                                       32821 non-null  float64
 4   key                                          32821 non-null  int64  
 5   loudness                                     32821 non-null  float64
 6   mode                                         32821 non-null  int64  
 7   speechiness                                  32821 non-null  float64
 8   acousticness                                 32821 non-null  float64
 9   instrumentalness                             32821 non-null  float64
 10

In [3]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression

X = spotify.drop(columns='track_popularity')
y = spotify.track_popularity

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [ ]:
#Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
#Number of features to consider at every split
max_features = ['auto', 'sqrt']
#Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
#Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
#Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
#Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid

forest_cv = GridSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_grid= {'n_estimators': n_estimators,
                 'max_features': max_features,
                 'max_depth': max_depth,
                 'min_samples_split': min_samples_split,
                 'min_samples_leaf': min_samples_leaf,
                 'bootstrap': bootstrap},
    n_jobs=-1,
    scoring='precision',
    cv=5,
)

forest_cv.fit(X_train, y_train)
forest_cv.score(X_test, y_test)

predeictions = forest_cv.predict(X_test)
error = mean_squared_error(y_true=y_test, y_pred=predeictions)
np.sqrt(error)

In [ ]:
forest = RandomForestRegressor(random_state=42)
forest.fit(X_train, y_train)
forest.score(X_test, y_test)

predeictions = forest.predict(X_test)
error = mean_squared_error(y_true=y_test, y_pred=predeictions)
np.sqrt(error)